In [2]:
import math
import random
import pandas as pd
import numpy as np
from scipy import stats
from scipy.integrate import quad


Task 1

In [3]:
# Функція для визначення рандомних координат для точок
def get_random_coords(lower_bound,upper_bound):
  return random.uniform(lower_bound,upper_bound)

In [4]:
def calculation(radius:float, exp_count:list, dots_numb:list):
  df = pd.DataFrame(index=exp_count, columns = dots_numb)
  #print(df)

# Зробимо велику таблицю де проведемо одразу усі досліди
  for index in exp_count:
    for test in dots_numb:
      dots_in_area = 0
      for i in range(1,test):

        x,y = get_random_coords(-radius, radius), get_random_coords(-radius, radius)
        # перевірка чи входить в коло точка
        in_area = pow(x,2) + pow(y,2) <= pow(radius,2)
        
        # За умови входження додаємо точку
        if in_area:
          dots_in_area += 1
          # if i == test-1:
      # Обраховуємо відношення точок в колі до всіх та обраховуємо приблизну площу
      aprx_area = round(pow(2*radius,2) * (dots_in_area/test),  2)
      df.at[index,test] = aprx_area


  df = df.apply(pd.to_numeric)
  
  mean_value = round(df.mean(),2)
  dispersion = round(df.var(),2)
  real_value = round(math.pi * pow(radius,2), 2)

  df.loc['Mean'] = mean_value
  df.loc['Dispersion'] = dispersion
  
  return df, mean_value, dispersion, real_value

In [5]:
# Визначаємо наші рядки та стовпці. Номер експерименту та кількість точок
exp_count = range(1,11)
dots_numb = [100,200,500,1000,2000,5000,10000,20000]

In [6]:
# Калькуляція
df1 = calculation(5, exp_count,dots_numb)

In [7]:
print('Area evaluation by Monte-Carlo trails numbers',
      '\n',df1[0],
      '\nReal Value:', df1[-1])

Area evaluation by Monte-Carlo trails numbers 
             100    200    500    1000   2000   5000   10000  20000
1           80.00  80.50  76.60   79.3  77.90  78.96  79.90  79.03
2           79.00  82.00  79.20   78.1  77.25  78.20  78.45  78.08
3           79.00  77.50  79.60   77.2  78.75  78.74  78.73  78.72
4           75.00  76.00  75.00   78.9  77.40  79.78  78.80  78.56
5           77.00  80.50  76.40   77.8  77.35  78.44  78.42  78.38
6           80.00  82.00  76.00   76.9  77.25  78.80  78.44  78.46
7           85.00  76.00  77.40   78.3  78.95  78.74  78.20  78.72
8           88.00  79.50  77.80   80.1  77.15  78.38  78.63  78.64
9           85.00  77.00  78.00   78.1  79.10  79.36  78.47  78.28
10          81.00  79.00  74.80   80.3  79.90  79.02  79.54  78.70
Mean        80.90  79.00  77.08   78.5  78.10  78.84  78.76  78.56
Dispersion  15.88   5.22   2.63    1.3   0.98   0.22   0.29   0.07 
Real Value: 78.54


Task 1 B

Lets take 95%-confidence interval

In [8]:
conf_level = 0.95

conf_intervals = pd.Series()
for dots_numb in df1[0].columns:
    # Обраховуємо параметри для обчислення
    sem = stats.sem(df1[0][dots_numb])
    mean = df1[0][dots_numb].mean()
    
    # Обраховуємо довірчі інтервали
    conf_interval = stats.t.interval(conf_level, len(df1[0][dots_numb][:len(exp_count)] )-1, loc=mean, scale=sem)
    conf_intervals[dots_numb] = conf_interval
    
    # print(dots_numb)
    # print(conf_interval)
print('Confridence intervals',
      '\n',conf_intervals)

Confridence intervals 
 100      (63.000602872391944, 87.9627304609414)
200      (58.87782118477047, 86.82551214856288)
500      (56.80834401948242, 84.94332264718425)
1000     (57.49788028999892, 86.63545304333438)
2000     (57.12344154537648, 86.22322512129018)
5000      (57.4661374475057, 87.11386255249431)
10000     (57.4234287276926, 87.01490460564071)
20000     (57.21993730873874, 86.8133960245946)
dtype: object


Task 2


In [104]:
pd.set_option('display.float_format', '{:.2f}'.format)

def integrand(x):
    numerator = -2 * x**3 + 5 * x + 11
    denominator = 2 * x**5 - 12 * x + 18

    # Check for invalid or zero values before calculation
    if numerator != 0 and denominator != 0:
        return np.cbrt(np.double(numerator)) / np.cbrt(np.double(denominator))
    else:
        return np.nan  # Ignore invalid values


# Межі інтегрування
a = 0
b = 7
# Створення списку кількості випадкових точок для експериментів
dots_numb = [100, 200, 500, 1000, 2000, 5000, 10000, 20000]

# Кількість експериментів
num_experiments = 10

# Створення DataFrame для збереження результатів
df2 = pd.DataFrame(index=range(1, num_experiments + 1))

# Проведення експериментів
for n in dots_numb:
    integral_estimates = []
    # points_inside_integral = []  # Відношення точок в інтегралі до всіх точок
    for _ in range(num_experiments):
        # Генерування випадкових значень x в межах інтегрування
        random_x = np.random.uniform(a, b, n)

        # Обчислення значень інтегранда для випадкових x з перевіркою на недійсність
        integrand_values = [integrand(x) for x in random_x if not np.isnan(integrand(x))]

        # Обчислення оцінки інтегралу
        if integrand_values:
            integral_estimate = np.mean(integrand_values) * (b - a)
            integral_estimates.append(integral_estimate)            
                  

    # Збереження результатів в DataFrame
    df2[f'{n}'] = integral_estimates


# Обчислення середнього та дисперсії для кожного експерименту
df2.loc['Mean'] = df2.mean()
df2.loc['Dispersion'] = df2.var()

# Виведення результатів
print(df2)
# print(sorted(integrand_values))


             100  200   500  1000  2000  5000  10000  20000
1           0.64 0.65  0.50  0.28  0.58  0.37   0.46   0.49
2           0.59 0.36  0.69  0.29  0.51  0.50   0.36   0.38
3           0.25 0.24 -0.00  0.52  0.37  0.54   0.47   0.41
4           1.10 0.75  0.11  0.58  0.39  0.30   0.38   0.43
5           0.73 0.48  0.49  0.58  0.37  0.42   0.46   0.36
6          -0.03 0.53  0.39  0.38  0.28  0.46   0.35   0.39
7           0.70 0.53  0.29  0.55  0.49  0.35   0.46   0.41
8           0.46 0.57  0.16  0.53  0.38  0.57   0.37   0.44
9           0.30 0.62 -0.04  0.52  0.48  0.36   0.47   0.38
10          0.67 0.42  0.37  0.51  0.49  0.40   0.37   0.43
Mean        0.54 0.52  0.30  0.47  0.44  0.43   0.41   0.41
Dispersion  0.09 0.02  0.05  0.01  0.01  0.01   0.00   0.00


In [101]:
from scipy.integrate import quad

def f(x):
    numerator = -2 * x**3 + 5 * x + 11
    denominator = 2 * x**5 - 12 * x + 18

    # Check for invalid or zero values before calculation
    if numerator != 0 and denominator != 0:
        return np.cbrt(np.double(numerator)) / np.cbrt(np.double(denominator))

integral, error = quad(f, 0, 7)
print("The integral is:", integral)
print("The absolute error is:", error)

The integral is: 0.4188381942
The absolute error is: 1.2780017950475663e-08


Task 3

In [114]:
import numpy as np

# Define the function to integrate
def f(x1, x2):
    return 3 * (x1**4) * (x2**5)

# Define the limits of integration
x1_bounds = (0, 5)
x2_bounds = (0, 4)

# Number of random points
df3 = pd.DataFrame(index=range(1,num_experiments+1))

for n in dots_numb:
    dintegral_estimates=[]
    for _ in range(num_experiments):

        # Генерування випадкових значень x в межах інтегрування
        random_x1 = np.random.uniform(*x1_bounds,n)
        random_x2 = np.random.uniform(*x2_bounds,n)

        # Обчислення значень інтегранда для випадкових x з перевіркою на недійсність
        dintegral_values = [f(x1,x2) for x1,x2 in zip(random_x1, random_x2) ]

        # Обчислення оцінки інтегралу
        if dintegral_values:
            dintegral_estimate = (x1_bounds[1]-x1_bounds[0])*(x2_bounds[1]-x2_bounds[0])*np.mean(dintegral_values)
            dintegral_estimates.append(dintegral_estimate)
        
    df3[f'{n}'] = dintegral_estimates

df3.loc['Mean'] = df3.mean()
df3.loc['Dispersion'] = df3.var()

print("The integral estimate is:", round(df3,2))

The integral estimate is:                        100            200            500           1000  \
1                968299.82      877726.47     1417120.05     1170791.31   
2               1699187.62      970720.13     1312434.41     1280175.71   
3                813143.70     1780317.62     1357925.88     1316164.33   
4               1239840.79     1045523.71     1448800.54     1180606.91   
5               1281097.13     1152722.19     1420949.04      942097.21   
6               1526104.50     1164901.10     1488589.82     1347337.51   
7               1658588.20     1543772.66     1088614.39     1378514.92   
8               1240839.59     1691297.90     1141453.46     1386621.47   
9                718482.19     1171205.79     1530733.03     1274281.02   
10              2240857.67     1501511.34     1415208.91     1139234.03   
Mean            1338644.12     1289969.89     1362182.95     1241582.44   
Dispersion 190203241353.93 89087372552.45 18722802940.53 16798805683.56   

In [102]:
from scipy.integrate import dblquad

# Define the function to integrate
def f(x1, x2):
    return 3 * (x1**4) * (x2**5)

# Define the limits of integration
x1_bounds = (0, 4)
x2_bounds = (0, 5)

# Compute the double integral
integral, error = dblquad(f, *x1_bounds, lambda x: x2_bounds[0], lambda x: x2_bounds[1])

print("The integral is:", round(integral,2))
print("The absolute error is:", error)

The integral is: 1280000.0
The absolute error is: 2.108585212300323e-08
